In [1]:
#!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import numpy as np
import sys
sys.path.insert(0,'../../scripts')

import pyccl as ccl
from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs
import pandas as pd
from astropy.io import fits
from astropy.table import Table

%matplotlib inline
catalog = 'skysim5000_v1.1.1'
skysim_cat = GCRCatalogs.load_catalog(catalog)
cosmo_ss  = skysim_cat.cosmology

In [2]:
cosmo = Nc.HICosmoDEXcdm()
reion = Nc.HIReionCamb.new () 
prim = Nc.HIPrimPowerLaw.new () 

cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new (2.0)

tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new (tf)

#psml = Nc.PowspecMLCBE.new ()
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()


cosmo.props.H0      = cosmo_ss.H0.value
cosmo.props.Omegab  = cosmo_ss.Ob0
cosmo.props.Omegac  = cosmo_ss.Odm0
cosmo.props.Omegax  = cosmo_ss.Ode0

cosmo.omega_x2omega_k ()
cosmo.param_set_by_name ("Omegak", 0.0)

prim.props.n_SA = cosmo_ss.n_s
print(cosmo_ss.sigma8,cosmo.sigma8(psf),cosmo.Omega_k0())

old_amplitude = math.exp (prim.props.ln10e10ASA)
prim.props.ln10e10ASA = math.log ((cosmo_ss.sigma8 / cosmo.sigma8(psf))**2 * old_amplitude)
print(cosmo_ss.sigma8, cosmo.sigma8(psf))

0.8 0.8490796381180891 0.0
0.8 0.8


In [3]:
#CosmoSim_proxy model
#M_0, z_0
theta_pivot = [3e14/0.71, 0.6]
#\mu_0, a_\mu^z, a_\mu^M
theta_mu = [3.19, -0.7, 2]
#\sigma_0, a_\sigma^z, a_\sigma^M
theta_sigma = [0.33, 0.,-0.08]

area = 439.78987
lnRl = 0.0
lnRu = 2.0
zl = 0.0
zu = 1.0

#Numcosmo_proxy model
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (zl, zu))

cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, lnRl, lnRu))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))


#cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (math.log(10)*np.log10(1e13),math.log(10)*np.log10(1e15)))

In [4]:
#Numcosmo Cluster Abundance

#First we need to define the multiplicity function here we will use the tinker
mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_linear_interp (True)
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
#Second we need to construct a filtered power spectrum 

hmf = Nc.HaloMassFunction.new(dist,psf,mulf)
hmf.set_area_sd(area)


ca = Nc.ClusterAbundance.new(hmf,None)
mset = Ncm.MSet.new_array([cosmo,cluster_m,cluster_z])

In [5]:
ncount = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
#ncount = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassNodist")
ca.prepare(cosmo,cluster_z,cluster_m)
mset = Ncm.MSet.new_array ([cosmo, cluster_z, cluster_m])

rng = Ncm.RNG.seeded_new (None, 42)

ncount.init_from_sampling (mset, area*((np.pi/180)**2), rng)
print (ncount.desc)

Cluster NCount resample unbinned. Generated 131104 from mean 1.7622e+05 (full). Mass proxy type `NcClusterMassAscaso', redshift proxy type `NcClusterRedshiftNodist'. Resampled in range [  0.0000,   1.0000] [1.00000000e+13, 1.00000000e+15] and area 439.7899 degrees square.


In [6]:


#Bins definition
z_edges = np.linspace(zl, zu, 5)
#proxy_edges = np.linspace(lnRl, lnRu, 4)
mass_edges = np.linspace(math.log(10)*13, math.log(10)*15, 4)

ncount.del_bins ()
for rich_l, rich_u in zip(mass_edges, mass_edges[1:]):
    for zl, zu in zip(z_edges,z_edges[1:]):
        lnM_obs_lb = Ncm.Vector.new_array ([rich_l])
        lnM_obs_ub = Ncm.Vector.new_array ([rich_u])
        z_obs_lb = Ncm.Vector.new_array ([zl])
        z_obs_ub = Ncm.Vector.new_array ([zu])
        
        ncount.add_bin (lnM_obs_lb, lnM_obs_ub, z_obs_lb, z_obs_ub)

ncount.bin_data ()



In [7]:
ncount.catalog_save ("ncount_ascaso2.fits", True)
ncount.catalog_load ("ncount_ascaso2.fits")

#ncount.catalog_save ("ncount_nodist.fits", True)
#ncount.catalog_load ("ncount_nodist.fits")

In [8]:
ncdata_fits = fits.open('ncount_ascaso2.fits')
#ncdata_fits.info()

ncdata_data =  ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table.columns
#ncdata_Table.sort('LNM_TRUE')

display(ncdata_Table)

Z_OBS,LNM_OBS,Z_TRUE,LNM_TRUE
float64,float64,float64,float64
0.9826436976154915,0.7969789589377404,0.9826436976154915,30.781168038976496
0.5656234907439457,0.4769605237164677,0.5656234907439457,30.660089305433786
0.6580973541649677,0.044217092035055805,0.6580973541649677,29.9520259094884
0.674313404266677,0.23258999507592445,0.674313404266677,30.007938230007394
0.47615085469970064,0.4649519197120849,0.47615085469970064,30.323157052162852
0.7612615579690223,1.0232154727016969,0.7612615579690223,30.39853929338206
0.44732157948676726,0.4191379681526473,0.44732157948676726,30.417877242238713
0.8895052279229502,0.10087840448363143,0.8895052279229502,30.282336625644366
0.8167259841927459,0.9181507414576693,0.8167259841927459,30.138319190267413


In [9]:
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (math.log(10)*np.log10(1e13),math.log(10)*np.log10(1e15)))
ncount_nodist = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassNodist")
ca.prepare(cosmo,cluster_z,cluster_m)
mset = Ncm.MSet.new_array ([cosmo, cluster_z, cluster_m])

hmf.set_area_sd (skysim_cat.sky_area)
ncount_nodist.set_lnM_true(Ncm.Vector.new_array(ncdata_Table['LNM_TRUE']))
ncount_nodist.set_z_true(Ncm.Vector.new_array(ncdata_Table['Z_TRUE']))

# Because we are using true masses and redshifts in this example,
# we replicate the true data in the 'observed' masses and redshift attributes
ncount_nodist.set_lnM_obs (Ncm.Matrix.new_array (ncdata_Table['LNM_TRUE'],1))
ncount_nodist.set_z_obs (Ncm.Matrix.new_array (ncdata_Table['Z_TRUE'],1))
ncount_nodist.props.area = skysim_cat.sky_area
ncount_nodist.true_data (True)
ncount_nodist.set_init(True)

print (ncount_nodist.desc)

ncount_nodist.catalog_save ("ncount_nodist.fits", True)
ncount_nodist.catalog_load ("ncount_nodist.fits")

ncdata_fits = fits.open('ncount_nodist.fits')
print (ncount_nodist.desc)






#ncdata_fits.info()

ncdata_data =  ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table.columns
ncdata_Table.sort('LNM_TRUE')

display(ncdata_Table)

None
None


Z_OBS,LNM_OBS,Z_TRUE,LNM_TRUE
float64,float64,float64,float64
0.596765394583555,29.933608038327165,0.596765394583555,29.933608038327165
0.45563382595055163,29.93362144982825,0.45563382595055163,29.93362144982825
0.8736772717559352,29.933629330711774,0.8736772717559352,29.933629330711774
0.7921318088524659,29.933629510165584,0.7921318088524659,29.933629510165584
0.41832253369004696,29.93364403468008,0.41832253369004696,29.93364403468008
0.14801427513659451,29.93365610430557,0.14801427513659451,29.93365610430557
0.9099856534922385,29.933656682441605,0.9099856534922385,29.933656682441605
0.6520576943943986,29.933668467508692,0.6520576943943986,29.933668467508692
0.7873763632926296,29.933677211955168,0.7873763632926296,29.933677211955168
